In [ ]:
import pandas as pd

from tclf.classical_classifier import ClassicalClassifier

In [ ]:
features_classical = [
    "TRADE_PRICE",
    "bid_ex",
    "ask_ex",
    "BEST_ASK",
    "BEST_BID",
    "price_all_lag",
]

features_size = [
    "TRADE_SIZE",
    "bid_size_ex",
    "ask_size_ex"
]

features_meta = ["QUOTE_DATETIME", "buy_sell"]

columns = [
    *features_classical,
    *features_size,
    *features_meta,
]

In [ ]:
gcs_loc = "gs://thesis-bucket-option-trade-classification/data/preprocessed/ise_supervised_none/"
X = pd.read_parquet(
        gcs_loc, engine="pyarrow", columns=columns
    )

X_meta = X[features_meta]
X = X.drop(columns=features_meta).rename({"TRADE_PRICE": "trade_price", "TRADE_SIZE": "trade_size", "BEST_ASK": "ask_best", "BEST_BID": "bid_best"}, axis=1)

In [ ]:
layers = [ # grauer (benchmark 2) 
        ("trade_size", "ex"),
        ("quote", "best"),
        ("quote", "ex"),
        ("depth", "best"),
        ("depth", "ex"),
        ("rev_tick", "all"),
    ]
clf = ClassicalClassifier(layers=layers, strategy="random")

X_meta["y_pred"] = clf.fit(X).predict(X)

In [ ]:
X_meta